In [1]:
import os
import random
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is
import lib.metrics
import lib.load_data
import lib.evaluate

from lib.evaluate import testModel
from lib.load_data import get_data
from lib.metrics import iou_metric

from glob import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2
from skimage import io
from skimage.transform import resize

import matplotlib.font_manager
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


print('TensorFlow version: {version}'.format(version=tf.__version__))
print('Keras version: {version}'.format(version=tf.keras.__version__))
print('Eager mode enabled: {mode}'.format(mode=tf.executing_eagerly()))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))

TensorFlow version: 2.6.0
Keras version: 2.6.0
Eager mode enabled: True
Num GPUs Available:  0
Num CPUs Available:  1


2022-12-13 12:41:11.056922: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/17.02.11/lib64/slurm:/cm/shared/apps/slurm/17.02.11/lib64:/cm/local/apps/gcc/6.3.0/lib:/cm/local/apps/gcc/6.3.0/lib64
2022-12-13 12:41:11.056994: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-13 12:41:11.057064: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (smcluster): /proc/driver/nvidia/version does not exist


# Metrics Prediction

In [2]:
img_height = 256
img_width = 256

path_test_levee = "../datasets/experiment_3/test"
test_images_leveecrack = sorted(next(os.walk(path_test_levee + "/images"))[2])

random.shuffle(test_images_leveecrack)
print(test_images_leveecrack)

image_leveecrack_path = path_test_levee + "/images"
mask_leveecrack_path = path_test_levee + "/masks"


X_test_levee, Y_test_levee = get_data(test_images_leveecrack, path_test_levee, img_height, img_width, train=True)

['pic_(82).jpg', 'pic_(45).jpg', 'pic_(12).jpg', 'pic_(21).jpg', 'pic_(157).jpg', 'pic_(40).jpg', 'pic_(15).jpg', 'pic_(103).jpg', 'pic_(135).jpg', 'pic_(132).jpg', 'pic_(49).jpg', 'pic_(76).jpg', 'pic_(64).jpg', 'pic_(150).jpg', 'pic_(145).jpg', 'pic_(92).jpg', 'pic_(25).jpg', 'pic_(8).jpg', 'pic_(123).jpg', 'pic_(122).jpg', 'pic_(29).jpg']
(21, 256, 256, 3)
Getting and resizing images ... 


100%|███████████████████████████████████████████| 21/21 [00:01<00:00, 18.21it/s]

Done!
(21, 256, 256, 3)


In [3]:
def evaluate_model_for_dataset(X_test, Y_test, model_name, dataset_name, experiment_name):
    img_height = 256
    img_width = 256
    
    result_folder_name = str(model_name) + "_" + str(dataset_name)
    root_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    model_name = model_name + "_best_model.h5"
    model_path = os.path.join(root_dir,  "models", str(experiment_name), str(model_name))
    tf.keras.backend.clear_session()
    %time
    best_model = tf.keras.models.load_model(model_path, compile=False)
    print(f'=========Loaded {model_name} for {dataset_name}===========')
    results = testModel(best_model, X_test, Y_test, 4, result_folder_name)
    print("==========Evaluation Completed============")

In [15]:
evaluate_model_for_dataset(X_test_levee, Y_test_levee, "unet", "levee", "experiment_3")
evaluate_model_for_dataset(X_test_levee, Y_test_levee, "multiresunet", "levee" , "experiment_3")
evaluate_model_for_dataset(X_test_levee, Y_test_levee, "attentionunet", "levee", "experiment_3")
evaluate_model_for_dataset(X_test_levee, Y_test_levee, "nestedunet", "levee", "experiment_3")
evaluate_model_for_dataset(X_test_levee, Y_test_levee, "iterlunet", "levee", "experiment_3")

CPU times: user 64 µs, sys: 0 ns, total: 64 µs
Wall time: 128 µs
=========Loaded unet_best_model.h5 for levee===========
6/6 [==============================] - 3s 364ms/step
0.28178063
0.41135877
0.6005029 0.37175754 0.45922154
Test Jacard Index : 0.28178063
Test Dice Coefficient : 0.41135877
Test iou_all :0.283764545143966
Test tversky : 0.40045375
Test f1_score : 0.45922154
==========Evaluation Completed============
CPU times: user 22 µs, sys: 11 µs, total: 33 µs
Wall time: 66.3 µs
=========Loaded multiresunet_best_model.h5 for levee===========
6/6 [==============================] - 6s 809ms/step
0.22503875
0.33582193
0.6208494 0.27700317 0.3830857
Test Jacard Index : 0.22503875
Test Dice Coefficient : 0.33582193
Test iou_all :0.22709006429119402
Test tversky : 0.31052116
Test f1_score : 0.3830857
==========Evaluation Completed============
CPU times: user 32 µs, sys: 0 ns, total: 32 µs
Wall time: 62.7 µs
=========Loaded attentionunet_best_model.h5 for levee===========
6/6 [==========

# Generate Inferences

### Method for Prediction

In [4]:
def prediction(test_ids, image_path, mask_path, unet, multiresunet, attnunet, nestedunet, proposed):
    ids = test_ids
    
    image_names, mask1, mask2, mask3, mask4, mask5,image_id, mask_id, has_mask = [], [], [], [], [], [],[], [], []
    
    image_path = os.path.abspath(image_path)
    mask_path = os.path.abspath(mask_path)
    
    # Iterate through each images in test dataset
    for n, i in tqdm(enumerate(ids), total=len(ids)):
        filename = i
        image_names.append(filename)
        i = image_path + '/' + i

        # Create a empty array of shape 1,256,256,1
        X = np.empty((1,256,256,3))
        # read the image
        img = io.imread(i)
        # Resize the image and coverting them to array of type float64
        img = cv2.resize(img, (256,256))
        img = np.array(img)
        
        # Strandardize image similar to that used in the training process
        img = img / 255
        
        X[0,] = img
        
        # Prediction of masks
        predict1 = unet.predict(X)
        predict2 = multiresunet.predict(X)
        predict3 = attnunet.predict(X)
        predict4 = nestedunet.predict(X)
        predict5 = proposed.predict(X)
        
        image_id.append(image_path + '/' + filename)
        mask_id.append(mask_path + '/' + format(filename.split('.')[0]+'.png'))
        has_mask.append(1)
    
        mask1.append(predict1)
        mask2.append(predict2)
        mask3.append(predict3)
        mask4.append(predict4)
        mask5.append(predict5)
            
            
    return pd.DataFrame({'file_name':image_names,'image_path': image_id, 'mask_path': mask_id,  'UNet_mask': mask1, 'MultiresUNet_mask': mask2, 'AttentionUNet_mask': mask3,
                         'NestedUNet_mask': mask4, 'IterLUNet_mask':mask5, 'has_mask': has_mask})

In [12]:
def iou_metric(y_true_in, y_pred_in):
    smooth = 1e-15
    intersection = y_true_in.ravel() * y_pred_in.ravel()
    union = y_true_in.ravel() + y_pred_in.ravel() - intersection

    iou = ((np.sum(intersection) + smooth)/(np.sum(union) + smooth))
    return iou

### Method for Visualization

In [13]:
def show_visualization(df_pred, filename, img_width, img_height):
    count = 0
    fig, axs = plt.subplots(8,7, figsize=(62, 78)) # (width, height)
    dim = (img_width,img_height)
    
    root_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    filepath = os.path.join(root_dir, "results", filename)
    
    for i in range(len(df_pred)):
        if df_pred.has_mask[i]==1 and count<8:
            
            #read levee crack image
            img = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img = cv2.resize(img, dim)
            axs[count][0].imshow(img)
            axs[count][0].set_axis_off() 
            axs[count][0].set_title('Original Image ' + str(count+1), fontsize=78)
    
    
            
            #read original mask and overlay original mask with levee crack image    
            mask = cv2.imread(df_pred.mask_path[i], cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, dim)
            
            img[mask==255] = (255,0,0)
            axs[count][1].imshow(img)
            axs[count][1].set_axis_off()
            axs[count][1].set_title('Ground Truth '+ str(count+1), fontsize=78)
            mask = np.array(mask, dtype = np.float64)
            mask = mask / 255
            
            #for U-Net (M1)
            pred_unet = np.array(df_pred.UNet_mask[i])
            iou_unet = iou_metric(mask, pred_unet )
            pred_unet = np.round(pred_unet,0).squeeze()
            img1 = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img1 = cv2.resize(img1, (256, 256))
            img1 = img1.squeeze()
            img1[pred_unet==1] = (255, 255, 255)
            axs[count][2].imshow(img1)
            axs[count][2].set_axis_off()
            axs[count][2].set_title('[IoU M1=' + str(round(iou_unet,2)) + ']', fontsize=78)
            
            
            #for MultiResUNet (M2)
            pred_multiresunet = np.array(df_pred.MultiresUNet_mask[i])
            iou_multiresunet = iou_metric(mask, pred_multiresunet )
            pred_multiresunet = np.round(pred_multiresunet,0).squeeze()
            img2 = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img2 = cv2.resize(img2, (256,256))
            img2 = img2.squeeze() / 255
            img2[pred_multiresunet==1] = (255, 255, 255)
            #axs[count][3].imshow((img2 * 255).astype(np.uint8))
            axs[count][3].imshow(img2)
            axs[count][3].set_axis_off()
            axs[count][3].set_title('[IoU M2=' + str(round(iou_multiresunet, 2)) + ']', fontsize=78)
            
            
            #for Attention UNet (M3)
            pred_attentionunet = np.array(df_pred.AttentionUNet_mask[i])
            iou_attentionunet = iou_metric(mask, pred_attentionunet )
            pred_attentionunet = np.round(pred_attentionunet,0).squeeze()
            img3 = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img3 = cv2.resize(img3, (256,256))
            img3 = img3.squeeze() / 255
            img3[pred_attentionunet==1] = (255, 255, 255)
            axs[count][4].imshow(img3)
            axs[count][4].set_axis_off() 
            axs[count][4].set_title('[IoU M3=' + str(round(iou_attentionunet, 2)) + ']', fontsize=78)
            
            
            #for UNet++ (M4)
            pred_nestedunet = np.array(df_pred.NestedUNet_mask[i])
            iou_nestedunet = iou_metric(mask, pred_nestedunet )
            pred_nestedunet = np.round(pred_nestedunet,0).squeeze()
            img4 = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img4 = cv2.resize(img4, (256, 256))
            img4 = img4.squeeze()
            img4[pred_nestedunet==1] = (255, 255, 255)
            axs[count][5].imshow(img4)
            axs[count][5].set_axis_off() 
            axs[count][5].set_title('[IoU M4=' + str(round(iou_nestedunet, 2)) + ']', fontsize=78)
            
            
            #for IterLUNet (M5)
            pred_inception = np.array(df_pred.IterLUNet_mask[i])
            iou_inception = iou_metric(mask, pred_inception )
            pred_inception = np.round(pred_inception,0).squeeze()
            img5 = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img5 = cv2.resize(img5, dim)
            img5 = img5.squeeze() 
            img5[pred_inception==1] = (0, 0, 255)
            axs[count][6].imshow(img5)
            axs[count][6].set_axis_off() 
            axs[count][6].set_title('[IoU M5=' + str(round(iou_inception, 2)) + ']', fontsize= 78)
            
            
            fig.subplots_adjust(top=0.94)
            fig.tight_layout()
            plt.savefig(filepath, format='png', facecolor="w", transparent=False)
        
            count +=1
        if (count==8):
            break
    fig.tight_layout() 
    plt.close()

### Method to load models

In [6]:
def load_model(model_name, experiment_name):
    result_folder_name = str(model_name)
    root_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    model_name = model_name + "_best_model.h5"
    model_path = os.path.join(root_dir,  "models", str(experiment_name), str(model_name))
    tf.keras.backend.clear_session()
    %time
    best_model = tf.keras.models.load_model(model_path, compile=False)
    print(f'=========Loaded {model_name}===========')
    return best_model

In [7]:
unet = load_model('unet', 'experiment_3')
multiresunet = load_model('multiresunet', 'experiment_3')
attention_unet = load_model('attentionunet', 'experiment_3')
nested_unet = load_model('nestedunet', 'experiment_3')
iterlunet = load_model('iterlunet', 'experiment_3')

CPU times: user 18 µs, sys: 19 µs, total: 37 µs
Wall time: 72 µs


2022-12-13 12:41:30.177871: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


=========Loaded unet_best_model.h5===========
CPU times: user 20 µs, sys: 21 µs, total: 41 µs
Wall time: 81.8 µs
=========Loaded multiresunet_best_model.h5===========
CPU times: user 17 µs, sys: 15 µs, total: 32 µs
Wall time: 62.7 µs
=========Loaded attentionunet_best_model.h5===========
CPU times: user 0 ns, sys: 45 µs, total: 45 µs
Wall time: 86.1 µs
=========Loaded nestedunet_best_model.h5===========
CPU times: user 17 µs, sys: 14 µs, total: 31 µs
Wall time: 61.3 µs
=========Loaded iterlunet_best_model.h5===========


In [8]:
df_all_preds_all = []
df_all_preds_all = prediction(test_images_leveecrack, os.path.abspath(image_leveecrack_path), os.path.abspath(mask_leveecrack_path), unet, multiresunet, attention_unet, nested_unet, iterlunet)

  0%|                                                    | 0/21 [00:00<?, ?it/s]2022-12-13 12:41:38.296673: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


100%|███████████████████████████████████████████| 21/21 [00:52<00:00,  2.52s/it]


In [14]:
df_for_visualization = df_all_preds_all.sample(frac = 0.98)
show_visualization(df_for_visualization, "iee_levee_inferences.png", img_width, img_height)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i